## 1. Execute imports

In [1]:
import pandas as pd
import numpy as np
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report

In [2]:
data= pd.read_csv("https://raw.githubusercontent.com/SahilSinhaLpu/Machine-Learning/master/Datasets/SomvervilleHappines.csv")
X = pd.read_csv('datasets/somerville_formula_c.csv', encoding = 'utf-8', delimiter=',')
y = data['D']
X

,X1,X2,X3,X4,X5,X6,X1*X2,X1*X3,X1*X4,X1*X5,...,X2*X3,X2*X4,X2*X5,X2*X6,X3*X4,X3*X5,X3*X6,X4*X5,X4*X6,X5*X6
0,3,3,3,4,2,4,9.0,9.0,12.0,6.0,...,9.0,12.0,6.0,12.0,12.0,6.0,12.0,8.0,16.0,8.0
1,3,2,3,5,4,3,6.0,9.0,15.0,12.0,...,6.0,10.0,8.0,6.0,15.0,12.0,9.0,20.0,15.0,12.0
2,5,3,3,3,3,5,15.0,15.0,15.0,15.0,...,9.0,9.0,9.0,15.0,9.0,9.0,15.0,9.0,15.0,15.0
3,5,4,3,3,3,5,20.0,15.0,15.0,15.0,...,12.0,12.0,12.0,20.0,9.0,9.0,15.0,9.0,15.0,15.0
4,5,4,3,3,3,5,20.0,15.0,15.0,15.0,...,12.0,12.0,12.0,20.0,9.0,9.0,15.0,9.0,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,5,3,3,1,3,5,15.0,15.0,5.0,15.0,...,9.0,3.0,9.0,15.0,3.0,9.0,15.0,3.0,5.0,15.0
139,5,2,3,4,2,5,10.0,15.0,20.0,10.0,...,6.0,8.0,4.0,10.0,12.0,6.0,15.0,8.0,20.0,10.0
140,5,3,3,4,4,5,15.0,15.0,20.0,20.0,...,9.0,12.0,12.0,15.0,12.0,12.0,15.0,16.0,20.0,20.0
141,4,3,3,4,4,5,12.0,12.0,16.0,16.0,...,9.0,12.0,12.0,15.0,12.0,12.0,15.0,16.0,20.0,20.0


## KNN

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

### Information gain

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [KNeighborsClassifier()]}]

In [7]:
%%time
knn_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_info.fit(X_train, y_train)

Wall time: 1.61 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x00000233D441CB80>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [KNeighborsClassifier()]}])

In [8]:
print(cross_val_score(knn_info, X, y, cv=10).mean())

0.5257142857142857


### Variance threshold

In [9]:
from sklearn.feature_selection import VarianceThreshold

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'classifier': [KNeighborsClassifier()]}]

In [11]:
%%time
knn_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
knn_variance.fit(X_train, y_train)

Wall time: 67.7 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}],
             scoring='accuracy')

In [12]:
print(cross_val_score(knn_variance, X, y, cv=10).mean())

0.5257142857142857


### Chi-Square

In [13]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', KNeighborsClassifier())])

In [15]:
search_space = [{'classifier': [KNeighborsClassifier()]}]

In [16]:
%%time
knn_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_chi.fit(X_train, y_train)

Wall time: 93.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x00000233D3240040>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}])

In [17]:
print(cross_val_score(knn_chi, X, y, cv=10).mean())

0.5538095238095238


### Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression

### Information gain

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': ['all']},
                {'classifier': [LogisticRegression()]}]

In [21]:
%%time
logistic_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_info.fit(X_train, y_train)

Wall time: 1.25 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x00000233D441CB80>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [LogisticRegression()]}])

In [22]:
print(cross_val_score(logistic_info, X, y, cv=10).mean())

0.5685714285714286


### Variance threshold

In [23]:
from sklearn.feature_selection import VarianceThreshold

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

search_space = [{'classifier': [LogisticRegression()]}]

In [25]:
%%time
logistic_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
logistic_variance.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 260 ms


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}],
             scoring='accuracy')

In [26]:
print(cross_val_score(logistic_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.5947619047619048


### Chi-Square

In [27]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', LogisticRegression())])

In [29]:
search_space = [{'classifier': [LogisticRegression()]}]

In [30]:
%%time
logistic_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_chi.fit(X_train, y_train)

Wall time: 130 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x00000233D3240040>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}])

In [31]:
print(cross_val_score(logistic_chi, X, y, cv=10).mean())

0.5623809523809523


## SVM

In [32]:
from sklearn.svm import SVC

### Information gain

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'selector__k': ['all']},
                {'classifier': [SVC(kernel='linear')]}]

In [35]:
%%time
svm_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_info.fit(X_train, y_train)

Wall time: 1.14 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x00000233D441CB80>)),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [SVC(kernel='linear')]}])

In [36]:
print(cross_val_score(svm_info, X, y, cv=10).mean())

0.5957142857142858


### Variance threshold

In [37]:
from sklearn.feature_selection import VarianceThreshold

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'classifier': [SVC(kernel='linear')]}]

In [39]:
%%time
svm_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
svm_variance.fit(X_train, y_train)

Wall time: 204 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'classifier': [SVC(kernel='linear')]}],
             scoring='accuracy')

In [40]:
print(cross_val_score(svm_variance, X, y, cv=10).mean())

0.6304761904761904


### Chi-Square

In [41]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', SVC())])

In [43]:
search_space = [{'classifier': [SVC()]}]

In [44]:
%%time
svm_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_chi.fit(X_train, y_train)

Wall time: 73.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x00000233D3240040>)),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()]}])

In [45]:
print(cross_val_score(svm_chi, X, y, cv=10).mean())

0.5633333333333334


## Naive Bayes

In [46]:
from sklearn.naive_bayes import GaussianNB

### Information gain

In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', GaussianNB())])

search_space = [{'selector__k': ['all']},
                {'classifier': [GaussianNB()]}]

In [49]:
%%time
nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

Wall time: 1.13 s


GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x00000233D441CB80>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [GaussianNB()]}])

In [50]:
print(cross_val_score(nb_info, X, y, cv=10).mean())

0.5204761904761905


### Variance threshold

In [51]:
from sklearn.feature_selection import VarianceThreshold

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

search_space = [{'classifier': [GaussianNB()]}]

In [53]:
nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}], scoring='accuracy')

In [54]:
print(cross_val_score(nb_variance, X, y, cv=10).mean())

0.5204761904761905


### Chi-Square

In [55]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', GaussianNB())])

In [57]:
search_space = [{'classifier': [GaussianNB()]}]

In [58]:
%%time
nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

Wall time: 53.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x00000233D3240040>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}])

In [59]:
print(cross_val_score(nb_chi, X, y, cv=10).mean())

0.5204761904761905


In [5]:
## Random Forest

from sklearn.ensemble import RandomForestRegressor

### Information gain

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'selector__k': ['all']},
                {'classifier': [RandomForestRegressor()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)
print('Random Forest -> Information gain')
print(cross_val_score(nb_info, X, y, cv=10).mean())

### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'classifier': [RandomForestRegressor()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print('Random Forest -> Chi-Square')
print(cross_val_score(nb_chi, X, y, cv=10).mean())


from sklearn.preprocessing import MinMaxScaler

from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [MLPClassifier()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

print('MLPClassifier -> Info')
print(cross_val_score(nb_info, X, y, cv=10).mean())

### Variance threshold

from sklearn.feature_selection import VarianceThreshold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', VarianceThreshold()),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

print('MLPClassifier -> Variance')
print(cross_val_score(nb_variance, X, y, cv=10).mean())


### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print('MLPClassifier -> Chi')
print(cross_val_score(nb_chi, X, y, cv=10).mean())

Random Forest -> Information gain
-0.36295069942679486
Random Forest -> Chi-Square
-0.32091881147416307


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

MLPClassifier -> Info


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

...

MLPClassifier -> Chi


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

0.5619047619047619


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
